In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
model = tf.keras.models.load_model("model/ncf_rec.keras")

2024-11-23 20:10:04.569691: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1184 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


In [4]:
df = pd.read_csv(f"data/item_ratings.csv")#.iloc[:1000,:]

In [6]:
user_id = 14524

seen_movies = df[df['userId'] == user_id]
movies_unneeded = df[df['movieId'].isin(seen_movies['movieId'])]
df_all = df.merge(movies_unneeded, on=['userId', 'movieId'], how="left", indicator=True)
unseen = df[df_all['_merge'] == 'left_only']
unseen_movie_ids = unseen['movieId'].unique()

In [8]:
unseen_movie_ids.shape[0]

5937

In [23]:
p = np.full((unseen_movie_ids.shape[0],2), user_id)
p[:,1] = unseen_movie_ids
p

array([[ 14524,    862],
       [ 14524,   8844],
       [ 14524,    949],
       ...,
       [ 14524,  45527],
       [ 14524,  49280],
       [ 14524, 111109]])

In [25]:
test_data = [p[:, 0], p[:, 1]]
out = model.predict(test_data)

  1/186 ━━━━━━━━━━━━━━━━━━━━ 37s 204ms/step

/home/dayan/projects/movie_recommendation/.conda/lib/python3.11/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_layer', 'input_layer_1']. Received: the structure of inputs=('*', '*')
  warnings.warn(
I0000 00:00:1732360618.735599  300021 service.cc:145] XLA service 0x737590004fa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732360618.735627  300021 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-11-23 20:16:58.740803: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-23 20:16:58.756451: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902
I0000 00:00:1732360618.893172  300021 device_compiler.h:188] Compiled cluster using XLA!  This line is logged a

186/186 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [ ]:
np.unique(out, return_counts=True)

(array([0.5009177 , 0.5276234 , 0.61134744, 0.6556127 , 0.8569486 ,
        0.9862061 , 0.9939889 , 0.99521565, 0.99991035, 0.9999838 ,
        0.99999356, 0.99999964, 1.        ], dtype=float32),
 array([   5,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1, 5921]))

In [33]:
top_n = np.argsort(out.flatten())[-10:][::-1]
top_n

array([5936, 1984, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982])

In [46]:
movies = pd.read_csv('data/movies_processed_cleaned.csv')
movies[movies['id'].isin(top_n)]

,Unnamed: 0,adult,genres,id,imdb_id,original_language,original_title,overview,popularity,production_companies,release_date,revenue,runtime,tagline,title
1846,2078,False,thriller,1978,tt0042994,en,Stage Fright,A struggling actress tries to help a friend pr...,4.329397,warner bros,1950-02-23,0.0,110.0,Love held its breath as sudden terror held the...,stage fright
6072,6926,False,"drama,romance",1976,tt0030287,en,Jezebel,"In 1850s Louisiana, the willfulness of a tempe...",11.277226,warner bros,1938-03-19,0.0,103.0,A Fearless Feminine Creature with a heart full...,jezebel
9785,11329,False,"horror,thriller",1975,tt0433386,en,The Grudge 2,"In Tokyo, a young woman is exposed to the same...",6.768804,"columbia pictures corporation,ghost house pict...",2006-10-13,39143839.0,102.0,"What Was Once Trapped, Will Now Be Unleashed",the grudge 2
10255,11864,False,"adventure,fantasy,action,thriller",1979,tt0486576,en,4: Rise of the Silver Surfer,The Fantastic Four return to the big screen as...,12.924254,"ingenious film partners,twentieth century fox ...",2007-06-13,289047763.0,92.0,Discover the secret of the Surfer.,fantastic 4: rise of the silver surfer
11172,12972,False,"drama,science fiction",1984,tt0048918,en,1984,"In a totalitarian future society, a man whose ...",6.113449,"columbia pictures corporation,holiday film pro...",1956-03-06,0.0,90.0,Big Brother is Watching.,1984
11491,13367,False,"mystery,horror,thriller",1977,tt1053859,en,The Grudge 3,"Jake, the sole survivor of The Grudge 2 massac...",10.510468,"ghost house pictures,ozla pictures,stage 6 films",2009-05-12,38610009.0,90.0,You can't escape the curse.,the grudge 3
